In [3]:
import os
import faiss
import json
import glob

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore import InMemoryDocstore
from langchain.schema import Document
from uuid import uuid4
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


In [4]:
os.environ["OpenAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model ="gpt-4o")

In [5]:
# folder_path = "food_recipe/"

# # 병합된 데이터를 저장할 리스트
# merged_data = []

# # 폴더 내 모든 JSON 파일 읽기
# for file_name in os.listdir(folder_path):
#     if file_name.endswith(".json"):  # JSON 파일만 처리
#         file_path = os.path.join(folder_path, file_name)
#         with open(file_path, 'r', encoding='utf-8') as file:
#             try:
#                 # JSON 데이터 불러오기
#                 data = json.load(file)
#                 merged_data.append(data)  # 병합 리스트에 추가
#                 print(f"Added data from {file_name}")
#             except json.JSONDecodeError as e:
#                 print(f"Error decoding {file_name}: {e}")

# # 병합된 데이터를 하나의 JSON 파일로 저장
# output_path = os.path.join(folder_path, "final_recipes_data.json")
# with open(output_path, 'w', encoding='utf-8') as output_file:
#     json.dump(merged_data, output_file, indent=4, ensure_ascii=False)

# print(f"병합된 JSON 데이터가 {output_path}에 저장되었습니다.")


In [6]:
# with open("food_recipe/food_recipes_1-68.json",'r',encoding='utf-8') as file:
#     data = json.load(file)

In [7]:
# document_recipe = [
#     Document(page_content=str(recipe), metadata={"요리명": recipe["요리명"],"요리재료" : recipe["요리재료"], "기본정보" : recipe["기본정보"], "조리순서" : recipe["조리순서"]})
#     for recipe in data
# ]

In [8]:
# document_recipe

In [9]:
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

In [10]:
# index  = faiss.IndexFlatL2(len(embeddings.embed_query("레시피")))

# recipe_store = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={}
# )

In [11]:
# uuids = [str(uuid4()) for _ in range(len(document_recipe))]

# recipe_store.add_documents(documents=document_recipe, ids = uuids)

In [12]:
# recipe_store.save_local("./food_db")
recipes = FAISS.load_local("./food_db" ,embeddings, allow_dangerous_deserialization=True)

In [13]:
retriever = recipes.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [14]:
# prompt = ChatPromptTemplate.from_template("""
# 너는 사용자가 물어보는 음식의 레시피를 자세하게 알려주는 요리사야.
# 친절하고 전문적인 말투로 응답해.
# 데이터에 포함되어 있는 레시피만을 제공하도록 해.
# 해당 음식이 데이터에 포함되어 있지 않다면 양해를 구하고 관련 데이터를 제공할 수 없음을 명시해줘.
# 대신 해당 음식과 비슷한 재료가 들어가있는 음식 리스트를 최대 5개 소개해줘.
# 기본적으로 해당 레시피의 원래 인분 수로 제공하고, 사용자가 원하는 인분 수에 따라 재료의 양을 계산해서 수정해줘.

# '''
# user : <음식명3> 을 만드는 법을 알려줘
# ai : <음식명3> 을 만드는 방법은 아래와 같습니다:
# - 요리의 이름
# - 재료
#     - 재료 1 (재료양)
#     - 재료 2 (재료양)
#     - 재료 3 (재료양)
# - 조리 시간
# - 인분 수
# - 요리 방법
#     - 첫번째, ~~
#     - 두번째, ~~
#     - 세번째, ~~
# '''

# '''
# user: 김치우동을 만드는법을 알려줘
# ai: 죄송합니다. 김치우동에 관련한 정보는 제가 가져올 수 없습니다. 
# 대신 김치우동과 비슷한 재료가 들어가는 음식들을 소개해드리겠습니다:
# - 요리명1
#   - 필요 재료
#   - 조리 시간: 15분
# - 요리명2
#   - 필요 재료
#   - 조리 시간: 10분
# ~~

# '''
# '''
# user : 1인분의 레시피를 알려줘
# ai : <음식명>의 레시피는 ~ 인분 기준입니다. 
# <음식명> 을 1인분으로 만드는 레시피는 다음과 같습니다:
# - 재료 : 1인분으로 수정된 재료 나열
# - 요리 방법 : 요리 방법 나열
# '''

# 다음과 같은 데이터를 학습해:\n{data}"
# 그리고 질문에 답해:\n{question}

# """)

In [17]:
prompt = ChatPromptTemplate.from_template("""
너는 사용자가 입력한 [재료]에 따라 관련 요리를 알려주는 요리사야. 
친절하고 전문적인 말투로 응답해.
이 재료들로 만들 수 있는 요리 최대 5개를 가져온 데이터에 있는 음식으로 알려줘.이 때 조리 시간과 사용되는 재료를 함께 알려줘.
사용자가 특정 요구 사항(예: "채식 요리", "저칼로리 요리")을 포함한 질문을 하면, 그에 맞는 요리 옵션을 제공해줘.
아래는 대화의 예시야
'''
user : 미역으로 만들 수 있는 음식 4개를 알려줘
ai : 미역이 재료로 포함되어 만들 수 있는 음식은 아래와 같습니다.
- 음식명1
    - 필요 재료 : 재료1, 재료2, 재료3
    - 조리 시간 : 20분
- 음식명2
    - 필요 재료 : 재료1, 재료2, 재료3
    - 조리 시간 : 15분
~~
이 요리들은 김치를 활용하여 맛있게 즐길 수 있는 대표적인 메뉴들입니다. 각 요리의 세부 조리 방법이 필요하시다면 말씀해 주세요.
'''
'''
user: OO 재료로 짧은 시간 안에 만들 수 있는 요리를 알려줘
ai: 다음은 OO 재료로 짧은 시간 안에 만들 수 있는 요리 입니다:
- 요리명1
  - 필요 재료
  - 조리 시간: 15분
- 요리명2
  - 필요 재료
  - 조리 시간: 10분
~~
'''
'''
user: 채식 요리 3개를 알려줘
ai: 채식 요리로 만들 수 있는 음식은 아래와 같습니다:
- 요리명1 (채식)
  - 필요 재료
- 요리명2 (채식)
  - 필요 재료
'''
'''
user: OO 재료로 만들 수 있는 저칼로리 음식이 있을까
ai: OO으로 만들 수 있는 저칼로리 음식은 아래와 같습니다:
- 요리명1 (~kcal)
  - 필요 재료
- 요리명2 (~kcal)
  - 필요 재료
'''
데이터에 포함되어 있는 레시피를 우선적으로 제공해.
해당 음식이 데이터에 포함되어 있지 않다면 양해를 구하고 관련 데이터를 제공할 수 없음을 명시해줘.
대신 해당 음식과 비슷한 재료가 들어가있는 음식 리스트를 최대 5개 소개해줘.
기본적으로 해당 레시피의 원래 인분 수로 제공하고, 사용자가 원하는 인분 수에 따라 재료의 양을 계산해서 수정해줘.
'''
user : <음식명3> 을 만드는 법을 알려줘
ai : <음식명3> 을 만드는 방법은 아래와 같습니다.
- 요리의 이름
- 재료
    - 재료 1 (재료양)
    - 재료 2 (재료양)
    - 재료 3 (재료양)
- 조리 시간
- 인분 수
- 요리 방법
    - 첫번째, ~~
    - 두번째, ~~
    - 세번째, ~~
'''
'''
user: 김치우동을 만드는법을 알려줘
ai: 죄송합니다. 김치우동에 관련한 정보는 제가 가져올 수 없습니다. 
대신 김치우동과 비슷한 재료가 들어가는 음식들을 소개해드리겠습니다:
- 요리명1
  - 필요 재료
  - 조리 시간: 15분
- 요리명2
  - 필요 재료
  - 조리 시간: 10분
~~
'''
'''
user : 1인분의 레시피를 알려줘
ai : <음식명>의 레시피는 ~ 인분 기준입니다. 
<음식명> 을 1인분으로 만드는 레시피는 다음과 같습니다
- 재료 : 1인분으로 수정된 재료 나열
- 요리 방법 : 요리 방법 나열
'''
다음과 같은 데이터를 학습해:\n{data}"
그리고 질문에 답해:\n{question}
""")

In [18]:
contextual_prompt = prompt

In [19]:
class DebugPassThrough(RunnablePassthrough):
    
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args,**kwargs)
        return output
    
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config = None, **kwargs):
        return {"data": inputs["data"], "question": inputs["question"]}
    
rag_chain_divide = {
    "data" : retriever,
    "question" : DebugPassThrough()

# contextual_prompt에 기능을 분리시키는 프롬프트 입력 (~라면 a를 반환, ~라면 b를 반환)
} | DebugPassThrough() | ContextToText()| contextual_prompt | model 


#위의 반환된 값에 따라서 레시피를 뽑을지, 음식을 추천 할 지에 대해서 나눔.
#그에 따라서  contextual_prompt의 변경이 필요함.
rag_chain_progress = {
    "data" : retriever,
    "question" : DebugPassThrough()
} | DebugPassThrough() | ContextToText()| contextual_prompt | model


In [20]:
while True:
    print("-----------------------------")
    
    query = input("질문을 입력해 주세요 (break 입력시 종료됩니다) : ")
    
    if query.lower() == "break":
        break
    
    response = rag_chain_divide.invoke(query)
    
    
    
    print("Question : ", query)
    print(response.content)

-----------------------------
Question :  볶음밥 요리를 알려줘
볶음밥 요리로 만들 수 있는 음식은 아래와 같습니다:

1. **계란볶음밥**
   - 필요 재료: 달걀 2개(100g), 밥 1공기(200g), 소금 약간, 간장 1큰술(15ml), 참기름 1큰술(15ml), 식용유 2큰술(30ml), 깨소금 1작은술(3g)
   - 조리 시간: 15분

2. **생청국장 마늘칩볶음밥**
   - 필요 재료: 낫토 2팩, 고추장아찌 3개, 셀러리 1/2대, 마늘 4쪽, 홍고추 1개, 간장 1큰술, 후춧가루 1/2작은술, 식물성 기름 3큰술, 소금 조금, 참기름 조금, 밥 3공기
   - 분량: 2인분 기준

이 요리들은 간단하면서도 맛있게 즐길 수 있는 볶음밥 메뉴들입니다. 각 요리의 세부 조리 방법이 필요하시다면 말씀해 주세요.
-----------------------------
Question :  볶음밥 요리 다섯개를 알려줘
볶음밥을 활용한 다양한 요리 옵션은 다음과 같습니다. 다만, 현재 데이터에 포함된 볶음밥 레시피 정보는 부족하여, 일반적으로 인기 있는 볶음밥 요리 몇 가지를 소개해드리겠습니다. 참고로 레시피의 자세한 내용은 제공하지 못하는 점 양해 부탁드립니다.

1. **김치볶음밥**
   - 필요 재료: 밥, 김치, 양파, 대파, 달걀, 참기름 등
   - 조리 시간: 약 20분

2. **새우볶음밥**
   - 필요 재료: 밥, 새우, 당근, 완두콩, 달걀, 간장 등
   - 조리 시간: 약 25분

3. **야채볶음밥**
   - 필요 재료: 밥, 당근, 양파, 피망, 브로콜리, 간장 등
   - 조리 시간: 약 20분

4. **닭고기볶음밥**
   - 필요 재료: 밥, 닭고기, 양파, 대파, 간장, 참기름 등
   - 조리 시간: 약 30분

5. **베이컨볶음밥**
   - 필요 재료: 밥, 베이컨, 양파, 대파, 달걀, 간장 등
   - 조리 시간: 약 20분

각 요리는 기본적인 볶음밥 조리법에 따라

## 문제점
 
 * 이전 대화 내용을 기억 못 함

* 나만의 문제일수도 있는데 대화를 좀 진행하고 나서 질문을 입력했는데도 출력 안 나오고 다시 질문 입력하라 했음
  * 다시 입력하면 한번에 각각의 질문과 출력이 함께 나옴

-----
## 해결 된 것
-  없는 음식 데이터를 들고 오지 않음
-  